# Data Wrangling:
This notebook imports the call spreadsheets as dataframes, stitches the dataframes together, then adds the sex and stimulation frequency columns. Lastly, it exports the final spreadsheet. <br>*-Directory should be structured accordingly: Detections/Male/1Hz or Detections/Female/50Hz, etc.*<br>*-Errors occur with the forward slash in the DeepSqueak generated column name 'Mean Power (dB/Hz)'. Should eventually rename it.*

### Imports, settings, and functions

In [1]:
## Install the neccessary imports and check working directory

import os
import pandas as pd
import seaborn as sns

In [2]:
print(os.getcwd())

/Users/zacharygutierrez/A_Drive/StowersLab


In [3]:
## Increase the width of all cells for just this notebook

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
## Function for combining spreadsheets within the same directory
## NOTE: Arguments must match the directory (e.g. /Male/1Hz -> 'male', '1')

def merge_sheets(directory, sex, stimulation_freq):
    """Combines all spreadsheets within the given
       directory and returns them as a dataframe."""
    
    path1 = directory
    path2 = sex.capitalize() + '/'
    path3 = stimulation_freq + 'Hz'
    directory = path1 + path2 + path3
    os.chdir(directory)
    
    df = pd.DataFrame() # start with an empty df, then append
    for filename in os.listdir(directory):    
        if filename.endswith(".xlsx"):
            sheet = pd.read_excel(filename)
            df = df.append(sheet, ignore_index=True)
        else:
            continue
            
    df['Sex'] = sex
    df['Stimulation'] = stimulation_freq
    return df

### Create the male/female call dataframes

In [5]:
directory = '/Users/zacharygutierrez/Dropbox (Scripps Research)/Detections/'

In [6]:
## Create the call dataframes for 1Hz
male_1 = merge_sheets(directory, 'male', '1')
female_1 = merge_sheets(directory, 'female', '1')

## Create the call dataframes for 5Hz
male_5 = merge_sheets(directory, 'male', '5')
female_5 = merge_sheets(directory, 'female', '5')

## Create the call dataframes for 10Hz
male_10 = merge_sheets(directory, 'male', '10')
female_10 = merge_sheets(directory, 'female', '10')

## Create the call dataframes for 25Hz
male_25 = merge_sheets(directory, 'male', '25')
female_25 = merge_sheets(directory, 'female', '25')

## Create the call dataframes for 50Hz
male_50 = merge_sheets(directory, 'male', '50')
female_50 = merge_sheets(directory, 'female', '50')

### Combine everything

In [7]:
## Create a list with all the dataframes
## Create a new dataframe by concatenating all in the list

male_list = [male_1, male_5, male_10, male_25, male_50]
male_df = pd.concat(male_list, sort=False)                # sort kword to suppress warning, keep current behavior...

female_list = [female_1, female_5, female_10, female_25, female_50]
female_df = pd.concat(female_list, sort=False)            # ...if not, the column order will be changed

total_calls = pd.concat([male_df, female_df], sort=False)

print('Male breakdown by stimulation freq:')
print(male_df['Stimulation'].value_counts(),'\n')
print('Female breakdown by stimulation freq:')
print(female_df['Stimulation'].value_counts())
total_calls.head()

Male breakdown by stimulation freq:
25    1796
50    1279
10     709
5      357
1        3
Name: Stimulation, dtype: int64 

Female breakdown by stimulation freq:
50    1295
25    1007
10     681
5      294
Name: Stimulation, dtype: int64


,ID,Label,Accepted,Score,Begin Time (s),End Time (s),Call Length (s),Principal Frequency (kHz),Low Freq (kHz),High Freq (kHz),Delta Freq (kHz),Frequency Standard Deviation (kHz),Slope (kHz/s),Sinuosity,Mean Power (dB/Hz),Tonality,Sex,Stimulation
0,1.0,USV,1.0,0.548342,28.025404,28.071404,0.046000,89.188849,83.057687,93.363607,10.305919,3.150801,52.311493,2.081640,-96.681192,0.184402,male,1
1,2.0,USV,1.0,0.589640,28.706204,28.752604,0.046400,89.713191,80.414176,100.426876,20.012701,5.043053,-195.647868,2.894629,-93.535514,0.183509,male,1
2,1.0,USV,1.0,0.519729,10.570844,10.620044,0.049200,84.372436,78.156246,95.992606,17.836360,3.810399,30.854385,2.817765,-94.192915,0.189857,male,1
0,1.0,USV,1.0,0.518870,5.744974,5.769120,0.024146,51.573108,49.577588,51.806874,2.229286,0.682904,75.120984,1.016520,-60.313195,0.545200,male,5
1,2.0,USV,1.0,0.910543,5.910829,5.926266,0.015437,54.259962,48.055880,60.426938,12.371058,3.520445,-753.507560,1.176979,-66.978746,0.550002,male,5


In [8]:
## Sanity check

print(len(male_df))
print(len(female_df))
print(len(total_calls))

assert len(male_df)+len(female_df)==len(total_calls)

#sns.heatmap(total_calls.isnull())  # this is a good way to visually check for null values

4144
3277
7421


In [9]:
## Create the 'lights_on' and 'lights_off' dataframes by filtering the 'Begin Time (s)' column

lights_on = total_calls[(total_calls['Begin Time (s)'] >= 5) & (total_calls['End Time (s)'] <= 10)]
lights_off = total_calls[(total_calls['Begin Time (s)'] > 10 )]

In [10]:
## Export the dataframes as a spreadsheet

os.chdir('/Users/zacharygutierrez/Desktop/catch')

total_calls.to_excel("total_calls.xlsx")
male_df.to_excel("male_df.xlsx")
female_df.to_excel("female_df.xlsx")
lights_on.to_excel("lights_on.xlsx") 
lights_off.to_excel("lights_off.xlsx")

# End